In [1]:
import pandas as pd #Basic packages for creating dataframes and loading dataset
import numpy as np

import matplotlib.pyplot as plt #Package for visualization

import re #importing package for Regular expression operations

from sklearn.model_selection import train_test_split #Package for splitting the data

from sklearn.preprocessing import LabelEncoder #Package for conversion of categorical to Numerical

from keras.preprocessing.text import Tokenizer #Tokenization
from tensorflow.keras.preprocessing.sequence import pad_sequences #Add zeros or crop based on the length
from keras.models import Sequential #Sequential Neural Network
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D #For layers in Neural Network
from keras.utils.np_utils import to_categorical

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Sentiment (1).csv to Sentiment (1).csv


In [7]:
import pandas as pd

# Load the dataset as a Pandas DataFrame
dataset = pd.read_csv("Sentiment (1).csv", header=0)


mask = dataset.columns.isin(['text', 'sentiment'])
data = dataset.loc[:, mask]




In [8]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

<ipython-input-8-cee1da567eb8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())
<ipython-input-8-cee1da567eb8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))


In [9]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ') 

In [10]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ') 
tokenizer.fit_on_texts(data['text'].values) 
X = tokenizer.texts_to_sequences(data['text'].values) 


In [11]:
X = pad_sequences(X) 

embed_dim = 
lstm_out = 

In [15]:
def createmodel():
    model = Sequential() #Sequential Neural Network
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1])) 
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)) 
    model.add(Dense(3,activation='softmax')) 
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy']) 
    return model


In [16]:
labelencoder = LabelEncoder() 
integer_encoded = labelencoder.fit_transform(data['sentiment']) 
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42) 

In [17]:
batch_size = 32 
model = createmodel() 
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2) 
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size) 
print(score)
print(acc)

291/291 - 61s - loss: 0.8218 - accuracy: 0.6444 - 61s/epoch - 209ms/step
144/144 - 2s - loss: 0.7730 - accuracy: 0.6640 - 2s/epoch - 12ms/step
0.7729825973510742
0.6640454530715942


In [18]:
print(model.metrics_names) #metrics of the model

['loss', 'accuracy']


In [19]:
model.save('sentimentAnalysis.h5') #Saving the model

In [20]:
from keras.models import load_model 
model= load_model('sentimentAnalysis.h5') 

In [21]:
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [22]:

sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence)
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0) 
sentiment_probs = model.predict(sentence, batch_size=1, verbose=2)[0] 

print(sentiment_probs)
if sentiment == 0:
    print("Neutral")
elif sentiment < 0:
    print("Negative")
elif sentiment > 0:
    print("Positive")
else:
    print("Cannot be determined")


1/1 - 0s - 278ms/epoch - 278ms/step
[0.66862655 0.10003673 0.23133668]
Neutral


In [23]:
from keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.model_selection import GridSearchCV 

model = KerasClassifier(build_fn=createmodel,verbose=2) 
batch_size= [10, 20, 40] 
epochs = [1, 2] 
param_grid= {'batch_size':batch_size, 'epochs':epochs} 
grid  = GridSearchCV(estimator=model, param_grid=param_grid) 
grid_result= grid.fit(X_train,Y_train) 

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) 

<ipython-input-23-6c99b49150f4>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters


744/744 - 113s - loss: 0.8287 - accuracy: 0.6472 - 113s/epoch - 152ms/step
186/186 - 3s - loss: 0.7438 - accuracy: 0.6703 - 3s/epoch - 15ms/step


744/744 - 108s - loss: 0.8189 - accuracy: 0.6508 - 108s/epoch - 145ms/step
186/186 - 3s - loss: 0.7650 - accuracy: 0.6724 - 3s/epoch - 14ms/step


744/744 - 114s - loss: 0.8244 - accuracy: 0.6488 - 114s/epoch - 154ms/step
186/186 - 2s - loss: 0.8078 - accuracy: 0.6541 - 2s/epoch - 13ms/step


744/744 - 112s - loss: 0.8279 - accuracy: 0.6464 - 112s/epoch - 151ms/step
186/186 - 2s - loss: 0.7759 - accuracy: 0.6695 - 2s/epoch - 12ms/step


...

Epoch 1/2
186/186 - 35s - loss: 0.8436 - accuracy: 0.6387 - 35s/epoch - 191ms/step
Epoch 2/2
186/186 - 26s - loss: 0.6978 - accuracy: 0.7061 - 26s/epoch - 138ms/step
47/47 - 1s - loss: 0.7313 - accuracy: 0.6767 - 946ms/epoch - 20ms/step


Epoch 1/2
186/186 - 36s - loss: 0.8469 - accuracy: 0.6308 - 36s/epoch - 196ms/step
Epoch 2/2
186/186 - 26s - loss: 0.6881 - accuracy: 0.7066 - 26s/epoch - 137ms/step
47/47 - 1s - loss: 0.7470 - accuracy: 0.6853 - 762ms/epoch - 16ms/step


Epoch 1/2
186/186 - 36s - loss: 0.8475 - accuracy: 0.6324 - 36s/epoch - 192ms/step
Epoch 2/2
186/186 - 25s - loss: 0.6922 - accuracy: 0.7053 - 25s/epoch - 136ms/step
47/47 - 1s - loss: 0.7630 - accuracy: 0.6851 - 1s/epoch - 24ms/step


Epoch 1/2
186/186 - 34s - loss: 0.8346 - accuracy: 0.6347 - 34s/epoch - 184ms/step
Epoch 2/2
186/186 - 29s - loss: 0.6735 - accuracy: 0.7145 - 29s/epoch - 154ms/step
47/47 - 1s - loss: 0.7676 - accuracy: 0.6819 - 846ms/epoch - 18ms/step


Epoch 1/2
233/233 - 43s - loss: 0.8254 - accuracy: 0.6432 - 43s/epoch - 183ms/step
Epoch 2/2
233/233 - 35s - loss: 0.6820 - accuracy: 0.7081 - 35s/epoch - 149ms/step
Best: 0.681374 using {'batch_size': 40, 'epochs': 2}
